In [40]:
import pandas as pd
import yfinance as yf
from IPython.display import display, Markdown

In [41]:
ticker_symbol = "AAPL"
expiration_index = 0

# Create and load the ticker
ticker = yf.Ticker(ticker_symbol)
expirations = ticker.options
selected_expiration = expirations[expiration_index]

print(f"Using expiration date: {selected_expiration}")

# Get option chain
option_chain = ticker.option_chain(selected_expiration)
calls = option_chain.calls
puts = option_chain.puts

# Normalize IV
def normalize_iv(df):
    df = df.copy()
    df['impliedVolatility'] = df['impliedVolatility'].astype(float)
    df['impliedVolatility'] = df['impliedVolatility'].apply(
        lambda x: x / 100 if x > 2 else x
    )
    return df

calls = normalize_iv(calls)
puts = normalize_iv(puts)

# Filter stirkes to get rid of outliers
current_price = ticker.info['regularMarketPrice']

calls = calls[(calls['strike'] >= current_price * 0.8) &
              (calls['strike'] <= current_price * 1.2)]
puts = puts[(puts['strike'] >= current_price * 0.8) &
            (puts['strike'] <= current_price * 1.2)]

# Further filtering of the dataframe
def filter_options(df):
    df = df[df['volume'] > 0]
    df = df[df['openInterest'] > 0]
    df = df[df['bid'] > 0]
    df['spread'] = df['ask'] - df['bid']
    df = df[df['spread'] / df['bid'] < 0.5]
    return df

filtered_calls = filter_options(calls)
filtered_puts = filter_options(puts)

# Results
display(Markdown('Filtered Calls'))
display(filtered_calls[['contractSymbol', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']].head())

display(Markdown('Filtered Puts'))
display(filtered_puts[['contractSymbol', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']].head())

print(f'Number of filtered calls: {len(filtered_calls)}')
print(f'Number of filtered puts: {len(filtered_puts)}')

Using expiration date: 2025-08-01


Filtered Calls

,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility
10,AAPL250801C00170000,170.0,40.17,38.60,40.15,5.0,56,0.020215
11,AAPL250801C00175000,175.0,35.00,33.70,35.15,5.0,39,1.817384
12,AAPL250801C00180000,180.0,29.10,28.85,29.30,10.0,390,1.450198
13,AAPL250801C00182500,182.5,26.95,26.10,27.70,5.0,12,1.470706
14,AAPL250801C00185000,185.0,23.99,23.85,24.20,20.0,145,1.218754


Filtered Puts

,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility
11,AAPL250801P00170000,170.0,0.02,0.02,0.03,5824.0,6589,1.070317
12,AAPL250801P00175000,175.0,0.04,0.03,0.04,2421.0,9586,0.968750
13,AAPL250801P00180000,180.0,0.06,0.06,0.07,641.0,4373,0.898439
14,AAPL250801P00182500,182.5,0.09,0.08,0.10,104.0,678,0.865236
15,AAPL250801P00185000,185.0,0.11,0.11,0.12,463.0,9564,0.820314


Number of filtered calls: 27
Number of filtered puts: 27
